# Sample Subscriber Nodes

The subscriber node is very similar to the Publisher node, and many of my nodes are both publishers and subscribers. 

The key difference is that the publisher and subscriber functions have a one to many relationship: there can be multiple subscribers subscribed to a topic, but there can only be one publisher.

A subscriber node receives the messages published by the subscriber nodes, up to the number of messages allowed in its queue.

In [20]:
import inspect
import rospy
from std_msgs.msg import String

# The Example Subscriber Node and Class Definition
In each package, located in the "viper/src" director of my git repository, there is a 'script' folder. Within this folder are the various nodes for that package (which usually are also a class definition of that node). We can take a look at the sample subscriber node I built within the "viper_toolkit" package.

In [2]:
from script import subscriber
subscriber_source = inspect.getsource(subscriber)
print (subscriber_source)

#!/usr/bin/env python3

import rospy
from std_msgs.msg import String

class SubscriberNode(object):
    
    def __init__(self):
        self.setup_ros()
        self.setup_subscriber()
        self.main_loop()
    
    def setup_ros(self):
        # Register our node
        rospy.init_node('sample_subscriber')
        
        # Create our message container. This allows us to retrieve the
        # previous message upon failure so that we do not disrupt 
        # the system loop in the event of a dropped message.
        self.message = String()
        
    def setup_subscriber(self):
        # Setup a subscription to the topic 'topic.' We will then receive
        # any messages sent to this topic.
        rospy.Subscriber(
            name = 'topic', 
            data_class = String, 
            callback = self.callback,
            queue_size = 1)
        
    def callback(self, msg):
        # This tells the node what to do when it receives a new message.
        # In this exam

Each of my class definitions will look similar, although will greatly vary in complexity. 

Lets take a look at the different parts of this node.

In [3]:
from script.subscriber import SubscriberNode

In [23]:
instance = SubscriberNode
instance.message = String("The time is now 1638136508.7424648")

In [5]:
def dissect(method):
    class_ = 'SubscriberNode'
    instance_ = 'instance'
    i = f'{instance_}.{method}'
    #i = f'instance.{method}'
    print(f'class {class_}(object):')
    print("")
    e = eval(i)
    print(inspect.getsource(e))

## 1. Initialize the Class Object
The __init__ class function executes at the time that the class in instantiated. We are telling the interpreter to first run the setup_ros() function, then run the setup_publisher() function, and finally to execute the main_loop() function.

In [6]:
dissect('__init__')

class SubscriberNode(object):

    def __init__(self):
        self.setup_ros()
        self.setup_subscriber()
        self.main_loop()



## 2. Setup ROS
We need to tell ROS that we would like to bring this node online; we are registering this kernel as "sample_subscriber". 

During this step we will also set the class "message" attribute using the class String.This allows us to retrieve the previous message upon failure so that we do not disrupt the system loop in the event of a dropped message. Note that the message types between the subscribers and the publishers must be the same.

In [7]:
dissect('setup_ros')

class SubscriberNode(object):

    def setup_ros(self):
        # Register our node
        rospy.init_node('sample_subscriber')
        
        # Create our message container. This allows us to retrieve the
        # previous message upon failure so that we do not disrupt 
        # the system loop in the event of a dropped message.
        self.message = String()



## 3. Setting up the Publisher
Next we will setup the subscriber API which allows a node to subscribe to a "Topic." This node will be receive all messages of the topic "topic", but will only hold on to the last message (queue_size=1).

When it receives a new message it will execute its class callback() function

In [8]:
dissect('setup_subscriber')

class SubscriberNode(object):

    def setup_subscriber(self):
        # Setup a subscription to the topic 'topic.' We will then receive
        # any messages sent to this topic.
        rospy.Subscriber(
            name = 'topic', 
            data_class = String, 
            callback = self.callback,
            queue_size = 1)



## 4. The Callback Function
The callback() function tells the node what to do once it receives a message. In this example we are asking the node to save the incoming message to the container we set up earlier. Rather than perform an action in the main loop, which is activated cyclically, we will have our action originate from this callback.

Thoughout my modules I use a variety of types of messages and callbacks to not only pass on data such as images and inference results, but to also publish "flags" or indicators that a node wants something.

In [9]:
dissect('callback')

class SubscriberNode(object):

    def callback(self, msg):
        # This tells the node what to do when it receives a new message.
        # In this example we are asking it to save the message to the 
        # container we instantiated earlier in the setup_ros() function.
        self.message = msg
        # Here, rather than perform our action in the main loop, which
        # is activated by cycle, we will have our action originate from
        # the callback, which is activated whenever a message is received
        # and hence the callback is called.
        self.action_loop()



We can take a look at the message received:

In [25]:
print ("Sample Message Received")
print (instance.message)

Sample Message Received
data: "The time is now 1638136508.7424648"


## 4. The Action Loop
In our example action loop we will take the most recent message and transform its case into capital letters. We access the data in the message through it message.data class property. We then output the results to the terminal, as well as return the results when this function is called.

In [10]:
dissect('action_loop')

class SubscriberNode(object):

    def action_loop(self):
        # We will have this node take a message, and then make it all 
        # capital letters. We first get the property we saved to 
        # self.message... in messages of type 'String' the message text 
        # is saved in the 'data' property.
        text = self.message.data
        # We then output the results to the log and the terminal
        rospy.loginfo(f'I heard {text}')



## 5. Main Loop
Like the publisher node, the main loop is the loop which will continually run while the node is online. Since all of our actions are occuring within the callback loop we do not have much in our main loop, other than the rospy.spin() function which keeps the kernal alive in the case that it is not actively processing data (but still waiting for messages).

In [11]:
dissect('main_loop')

class SubscriberNode(object):

    def main_loop(self):

        # spin() simply keeps python from exiting until this node is stopped
        rospy.spin()



## Results

Our resulting output looks like this.